In [1]:
from lib.algorithms import Q_learn_freetime, Q_learn

In [2]:
import hydra
hydra.initialize(config_path='configs')
cfg = hydra.compose(config_name='config')


In [3]:
from lib.gym_windy_gridworld import WindyGridworld
env = WindyGridworld(
        height=cfg.env.height, 
        width=cfg.env.width, 
        rewards=list(cfg.env.rewards), 
        wind=cfg.env.wind, 
        start=cfg.env.start, 
        allowed_actions=list(cfg.env.allowed_actions), 
        reward_terminates_episode=cfg.env.reward_terminates_episode
    )
    

In [4]:
cfg

{'initializations': ['pessimistic'], 'random_initialization_seed': None, 'num_runs': 5, 'env': {'height': 20, 'width': 11, 'rewards': [[1, 0, 0], [10, 0, 11]], 'wind': True, 'start': 'random', 'allowed_actions': ['L', 'R', 'C'], 'reward_terminates_episode': True}, 'baseline': {'discount': 0.98, 'alpha': 0.01, 'num_steps': 10000, 'epsilon': 0.05, 'show_rewards': True, 'show_q': False, 'show_trajectory': True}, 'freetime': {'num_steps': 10000, 'epsilon': 0.05, 'discount': 0.98, 'alpha': 0.01, 'alpha_f': 0.01, 'tolerance': 0.0, 'show_rewards': True, 'show_q': False, 'show_f': True, 'show_f_actions': ['min'], 'show_trajectory': True}, 'trajectory_maps': {'num_plots': 2}, 'q_plots': {'vmin': 0.8, 'vmax': 1}, 'f_plots': {'vmin': None, 'vmax': None}, 'plot_freetime_vs_baseline_same_table': True}

In [5]:
cfg.freetime.num_steps = 100000
cfg.baseline.num_steps = 100000
cfg

{'initializations': ['pessimistic'], 'random_initialization_seed': None, 'num_runs': 5, 'env': {'height': 20, 'width': 11, 'rewards': [[1, 0, 0], [10, 0, 11]], 'wind': True, 'start': 'random', 'allowed_actions': ['L', 'R', 'C'], 'reward_terminates_episode': True}, 'baseline': {'discount': 0.98, 'alpha': 0.01, 'num_steps': 100000, 'epsilon': 0.05, 'show_rewards': True, 'show_q': False, 'show_trajectory': True}, 'freetime': {'num_steps': 100000, 'epsilon': 0.05, 'discount': 0.98, 'alpha': 0.01, 'alpha_f': 0.01, 'tolerance': 0.0, 'show_rewards': True, 'show_q': False, 'show_f': True, 'show_f_actions': ['min'], 'show_trajectory': True}, 'trajectory_maps': {'num_plots': 2}, 'q_plots': {'vmin': 0.8, 'vmax': 1}, 'f_plots': {'vmin': None, 'vmax': None}, 'plot_freetime_vs_baseline_same_table': True}

In [6]:
from lib.algorithms import build_q_table

def grid_search(initialization):
    
    results = {}
    for tolerance in [-0.01, -0.001, 0.0, 0.001, 0.01, 0.1, 0.5]:
        
        results[tolerance] = []
        
        for exp in range(10):
            
            Q = build_q_table(
                (env.height, env.width),                
                env.action_space.n, 
                initialization = initialization,
                seed = cfg.random_initialization_seed # type: ignore
            )
            
            Q, F, rewards, _ = Q_learn_freetime(
                env, 
                Q, 
                cfg.freetime.num_steps, 
                cfg.freetime.epsilon, 
                cfg.freetime.discount, 
                cfg.freetime.alpha, 
                cfg.freetime.alpha_f, 
                cfg.freetime.tolerance
            )
            
            results[tolerance].append(rewards[-1])
    
    results['baseline'] = []
    for exp in range(10):
        
        Q = build_q_table(
                (env.height, env.width),                
                env.action_space.n, 
                initialization = initialization,
                seed = cfg.random_initialization_seed # type: ignore
            )
        
        Q, rewards = Q_learn(
                env, 
                Q, 
                cfg.baseline.num_steps, 
                cfg.baseline.epsilon, 
                cfg.baseline.discount, 
                cfg.baseline.alpha
            )
        
        results['baseline'].append(rewards[-1])
        
    results = {k: sum(v)/len(v) for k, v in results.items()} 
    return results


In [7]:
results = grid_search('pessimistic')
results

100%|██████████| 100000/100000 [00:02<00:00, 36933.77it/s]


{-0.01: 23871.5,
 -0.001: 23750.2,
 0.0: 20382.5,
 0.001: 21955.1,
 0.01: 24422.6,
 0.1: 19739.3,
 0.5: 21550.5,
 'baseline': 7358.6}

In [8]:
results = grid_search('random')
results

100%|██████████| 100000/100000 [00:02<00:00, 38124.12it/s]


{-0.01: 17718.9,
 -0.001: 13085.5,
 0.0: 18547.2,
 0.001: 8481.6,
 0.01: 13765.7,
 0.1: 13707.3,
 0.5: 9931.6,
 'baseline': 9698.4}

In [9]:
results = grid_search('optimistic')
results

100%|██████████| 100000/100000 [00:02<00:00, 38020.43it/s]


{-0.01: 37473.2,
 -0.001: 38243.5,
 0.0: 38123.0,
 0.001: 38170.4,
 0.01: 38460.6,
 0.1: 37514.3,
 0.5: 37915.3,
 'baseline': 38829.1}